In [ ]:
%matplotlib inline

# Import modules

In [ ]:
from skan.pre import threshold
from tqdm.notebook import tqdm
from skimage.io import imread
import os, re, glob
from scipy.spatial.distance import cdist
import pandas as pd
from math import sqrt
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import napari
import csv
import pyclesperanto_prototype as cle
import nrrd
from operator import itemgetter
import seaborn as sns
import expansion_analysis_function as expf
%gui qt

In [ ]:
#make pdfs workable with illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42 

# Insert pixel size and calculate relevant factors

Here, ``pixel_size_x`` and ``pixel_size_z`` are the pixel sizes you get from the image metadata. They need to be adjusted accordingly to recieve the right scaling factor ``size_z`` as well as the right proportions for the analysis later on.
Additionally, in this section, the minimal distance between two Brp puncta ``min_distance`` as well as the permissive distance between Brp and GFP signal ``distance_gfp_brp`` are transformed into pixels. The respective values are taken from literature (Gao et al. 2019 and Fouquet et al. 2009).

In [ ]:
#get pixel sizes and calculate the scaling factor size_z
pixel_size_x = 0.06 #adjust for your recording 
pixel_size_z = 0.333

size_z = float("{:.2f}".format(pixel_size_z/ pixel_size_x))

min_distance = int(0.2/pixel_size_x) #200 nm distance between two Brp puncta
distance_brp_gfp  = int(0.155*2/pixel_size_x) #155 nm distance between Brp and GFP signal

# Load raw and preprocessed files
In this section we load the data of the raw and preprocessed image files as well as already generated .csv files.
``mainpath`` is the data path where all the data is located, including:
- raw .tif data,  ``image_file`` is a list with all the .tif files in the directory.
- .nrrd files which were generated by preprocessing the data with the VVD Viewer (https://github.com/JaneliaSciComp/VVDViewer). During the preprocessing step, we select the area of interest, e.g. the dendritic part of Tm9 neurons for both the GFP and the Brp channels. Furthermore, during this step we reduce the noise level of the image. ``nrrd_path`` is a list with all the .nnrd files saved in this directory.
- During the analysis with this code, .csv files will be generated with the roi coordinates and the closest Brp puncta. ``csv_files`` is a list where all the .csv files of the directory are listed.

In [ ]:
mainpath = r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm16_Tm9\2023_04_21_brain3' #change your base path here
image_file = np.sort(glob.glob(os.path.join(mainpath,'*.tif')))
nrrd_path = np.sort(glob.glob(os.path.join(mainpath, '*.nrrd')))
csv_file = np.sort(glob.glob(os.path.join(mainpath, '*.csv')))
image_file

### Get raw data
Depending on how one generated the .tif file from a .lif microscope file, there are either one or two .tif files (channels seperated or channels in one file). To load the image files you can use one of the two code pieces below. In both cases, please choose the .tif file you want to use from the ``image_file`` list. Next, we can slice the array to get the part of the image we want to use for analysis (Splitting the image in smaller parts is important if your computer cannot handle big data). <br>The coordinates are: [z, channel, y, x] or [z, y, x].

In [ ]:
#this code piece is for a file with all channels in one .tif file
#if you have images with multiple channels, you need to choose which channel to take: both_channels[z, channel, y, x]
both_channels = skimage.io.imread(image_file[0])
image_part_brp = both_channels[:,1,353:1870,:1388]
image_part_gfp = both_channels[:,0,353:1870,:1388]

#this code piece is for .tif files which have only one channel 
# gfp_image = skimage.io.imread(image_file[0])
# brp_image = skimage.io.imread(image_file[1])
# image_part_gfp = gfp_image[:, 675:1871, :1534] #slice your image in all 3 dimensions [z, y, x]
# image_part_brp = brp_image[:, 675:1871, :1534]


### Get preprocessed data
First, we want to load the preprocessed .nrrd data, get it into the same structure as the raw image and slice it the same way to get the same image parts for the raw and preprocessed data.

In [ ]:
brp_mask_vvd = nrrd.read(nrrd_path[0])
gfp_mask_vvd = nrrd.read(nrrd_path[1])

brp_transpose = brp_mask_vvd[0].T
brp_correct_shape = brp_transpose[:,353:1870,:1388]
#brp_correct_shape.shape

gfp_transpose = gfp_mask_vvd[0].T
gfp_correct_shape = gfp_transpose[:,353:1870,:1388]
gfp_correct_shape.shape #check if the shape is correct

Next, we use the preprocessed data to make a mask for each of the Brp and GFP channels and fill it with the real values from the raw image.

In [ ]:
region_props_brp = expf.thresholding_and_props(brp_correct_shape, threshold=1)
new_mask_brp = expf.make_mask (image_part_brp, region_props_brp, region_count=1)
brp_masked_stack = image_part_brp*(new_mask_brp*1)

region_props_gfp = expf.thresholding_and_props(gfp_correct_shape)
new_mask_gfp = expf.make_mask (image_part_gfp, region_props_gfp, region_count=1)
gfp_masked_stack = image_part_gfp*(new_mask_gfp*1)

# Visualize data in napari

In [ ]:
#open a napari viewer
%gui qt
viewer = napari.Viewer()

In [ ]:
#visualize raw data
viewer.add_image(image_part_gfp, scale=(size_z, 1, 1),blending='additive',colormap='green')
viewer.add_image(image_part_brp, scale=(size_z, 1, 1),blending='additive',colormap='magenta')

#visualize preprocessed and masked data
viewer.add_image(brp_masked_stack, scale=(size_z, 1, 1),blending='additive',colormap='magenta')
viewer.add_image(gfp_masked_stack, scale=(size_z, 1, 1),blending='additive',colormap='gray')

# Drawing manual ROIs and finding synapses between Tm9 and their presynaptic partners
Depending on the expression pattern of the presynaptic partner, we can either draw ROIs based on the Brp or the GFP signal.
- A) For C3 we can use the Brp channel to get the ROIs.
- B) For Tm16 and Dm12 we can use the GFP channel.

## A) Using Brp expression in C3 cells to manually draw ROIs and calculate closest puncta to Tm9

### 1. Draw the ROIs and save them into a .csv file
- In the Assistant (clEsperanto) plugin of napari one can use the **Projection** tool to create a maximum_z_projection of the ``brp_masked_stack`` image (or use the code piece below).
- Use the plugin **regions of interest (napari-roi)** (https://github.com/BodenmillerGroup/napari-roi) to create manual ROIs:
    - Open the plugin
    - Create a new shapes layer by clicking on the **new shapes layer** button
    - Press the **add ellipses** button
    - Draw a ROI around each Tm9 dendrite
- After drawing all ROIs, save the coordinates of the ROIs in a .csv file:
    - Go to *File* --> *Save Selected Layers* 
    - Name file and save as .csv in the same directory as the image files (see ``mainpath``)

In [ ]:
# maximum z projection
image1_mzp = cle.maximum_z_projection(brp_masked_stack)
viewer.add_image(
    image1_mzp, name='Result of maximum_z_projection (clesperanto)')


### 2. Use the newly generated ROIs to extract all Brp puncta within each region
- Rerun the cell in which ``mainpath`` and ``csv_file`` is defined to include the roi.csv file.
- Choose the correct .csv file from the ``csv_file`` list.

In [ ]:
roi_coords = pd.read_csv(csv_file[0])
#csv_file

- ``min_distance`` is the minimal distance between two local maxima, meaning how close two Brp puncta can be.
- ``thresholds`` should be used to adjust the luminance threshold, everything below this value is considered noise. With this parameter you can make sure to only detect a real signal.

In [ ]:
rois_df = expf.get_all_brps_in_ROI (brp_masked_stack, roi_coords, image_part_brp, min_distance, thresholds = 137, size_z = size_z)

In [ ]:
csv_save_roi = os.path.join(r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm9_C3', '2023_02_15_Tm9_C3_brain2_ol2_1_brp_ROIs_y890_end_x_2020_all_brp.csv')
rois_df.to_csv(csv_save_roi)

- Display all Brp puncta for each ROI color coded by column.

In [ ]:
expf.plot_colourcoded_puncta_napari (rois_df, viewer)

### 3. Find synapses between Brp puncta in each ROI and Tm9 signal
Now that we have Brp puncta for each ROI, we can calculate the distance between them and every coordinate in a binary GFP mask and threshold the distances by the maximal allowed space between pre- and postsynapses. This way, we can find synapses between Tm9 and C3 for each column.

In [ ]:
distance_df = expf.get_closest_brp_from_brp_clusters (new_mask_gfp, size_z, rois_df, distance_brp_gfp)

In [ ]:
csv_save_roi = os.path.join(r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm9_C3', '2023_02_15_Tm9_C3_brain2_ol2_1_brp_ROIs_y890_end_x_2020_rois_closest_brp.csv')
distance_df.to_csv(csv_save_roi)

**Please continue with the section: Quantification of synapses across flies.**

## B) Using GFP expression in Tm9 cells to manually draw ROIs and use them for synapse quantification

### 1. Draw the ROIs and save them into a .csv file
- In the Assistant (clEsperanto) plugin of napari one can use the **Projection** tool to create a maximum_z_projection ``gfp_masked_stack`` image (or use the code piece below).
- Use the plugin **regions of interest (napari-roi)** (https://github.com/BodenmillerGroup/napari-roi) to create manual ROIs:
    - Open the plugin
    - Create a new shapes layer by clicking on the **new shapes layer** button
    - Press the **add ellipses** button
    - Draw a ROI around each Tm9 dendrite
- After drawing all ROIs, save the coordinates of the ROIs in a .csv file:
    - Go to *File* --> *Save Selected Layers* 
    - Name file and save as .csv in the same directory as the image files (see ``mainpath``)

In [ ]:
# maximum z projection
image1_mzp = cle.maximum_z_projection(gfp_masked_stack)
viewer.add_image(
    image1_mzp, name='Result of maximum_z_projection (clesperanto)')


### 2. Calculating Brp local maxima to get coordinates of all Brp puncta
- Here, the ``min_distance`` is the minimal distance between two local maxima, meaning how close two Brp puncta can be.
- ``thresholds`` should be used to adjust the luminance threshold, everything below this value is considered noise. With this parameter you can make sure to only detect a real signal.

In [ ]:
brp_max = expf.calculate_local_max(brp_masked_stack, min_distance, thresholds = 169, filename = 'brp', size_z = size_z) #brp_mask_part, brp_masked_stack
all_results = pd.concat(brp_max)
brp_coords = np.array(all_results[[0,1,2]])
brp_puncta = expf.points_for_napari(brp_coords)
viewer.add_points((np.array(brp_puncta)).T, scale=(1, 1, 1), size=14, face_color='blue')

### 3. Use newly generated ROI coordinates to find synapses between Tm9 and an input neuron
In this section, we calculate the distance between each Brp puncta and each coordinate in every ROI, finding the pair with the minimum distance and thresholding it by the ``distance_brp_gfp`` value. If the distance between the ROI coordinate and Brp puncta is smaller than this threshold, we consider that the Tm9-Brp pair is forming a synapse.

- Rerun the cell in which ``mainpath`` and ``csv_file`` is defined to include the roi.csv file.
- Choose the correct .csv file from the ``csv_file`` list.

In [ ]:
csv_file

In [ ]:
roi_coords = pd.read_csv(csv_file[0])
distance_df = expf.get_closest_brp_per_roi (new_mask_gfp, roi_coords, image_part_gfp, brp_coords, distance_brp_gfp, size_z)
distance_df

- Save the ``distance_df`` DataFrame which stores the ROI (cluster) information, the coordinates of the GFP and Brp pair forming a synapse and the distance between the two (in pixel).

In [ ]:
csv_save_roi = os.path.join(r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm16_Tm9', '2023_04_21_Tm9_Tm16_brain3_ol2_1_y_350_1870_x_1388_brp_169.csv')
distance_df.to_csv(csv_save_roi)

# Quantification of synapses across flies
In the .csv file created from ``distance_df`` we stored the information about all detected synapses between Tm9 and its presynaptic partner. We can recall this information to quantify the number of synapses per Tm9 neuron (ROI) within a fly, as well as across multiple flies.

- Read in the files:
    - .csv with the information of found synapses (``distance_df``)
    - .csv with ROI coordinates (``roi_coords``)

In [ ]:
distance_df=pd.read_csv(csv_file[1])
distance_df = distance_df.drop(columns= 'Unnamed: 0')
roi_coords = pd.read_csv(csv_file[0])

- Quantify the number of synapses:
    - If you used the Brp channel to draw ROIs, please select ``roi_type = 'brp'``, if you used the GFP channel, please use ``roi_type = 'gfp'``.
    - Additionally in this step, we display the synapses in napari with a color code. Synapses of the same cluster/ ROI/ neuron will have the same color.

In [ ]:
%gui qt
viewer = napari.Viewer() #comment out if you have already a viewer open
#if you used the Brp channel to draw ROIs, please select roi_type = 'brp', if you used the GFP channel, please use roi_type = 'gfp'
nearest_synapse_count = expf.get_synapse_counts (roi_coords, distance_df, viewer, rois_df = None, brp_coords = brp_coords, roi_type ='gfp', point_size=15)

- Create a DataFrame to store the info.

In [ ]:
df_closest_brp_curr_flies = pd.DataFrame({'fly_ID':5, 'cluster': range(len(nearest_synapse_count)), 'counts': nearest_synapse_count, 'method' : 'exm'})
df_closest_brp_curr_flies

- Run this cell only for the first fly you have. This will be the shared DataFrame for all flies.

In [ ]:
df_closest_brp_all_flies = df_closest_brp_curr_flies
#df_closest_brp_all_flies

- Add the quantification of different flies into one DataFrame. For this, you need to re-run the code until here for images of other flies.
- Save the DataFrame at any time inbetween flies.

In [ ]:
df_closest_brp_all_flies = pd.concat([df_closest_brp_all_flies, df_closest_brp_curr_flies])

In [ ]:
csv_save_all_flies = os.path.join(r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm9_Tm16', 'counts_all_flies_Tm9_Tm16.csv')
df_closest_brp_all_flies.to_csv(csv_save_all_flies)

# Include EM data 
Here, we include the synapse counts of the FAFB connectome dataset with the FlyWire analysis. The data sheet with the synaptic counts from FlyWire were provided by another script. We can directly load the data and combine the counts we are interested in with the ExM data list.
For this we need the excel files ``Tm9_FAFB_L`` and ``Tm9_FAFB_R`` which can be found in the *EM* --> *processed_data* folder.

In [ ]:
csv_path = r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm16_Tm9'
csv_list = np.sort(glob.glob(os.path.join(csv_path, '*.csv')))
df_closest_brp_all_flies = pd.read_csv(csv_list[1])
#df_closest_brp_all_flies= df_closest_brp_all_flies.drop(columns= 'Unnamed: 0')
csv_list

In [ ]:
em_path = r'C:\Users\jcornean\PhD\expansion_microscopy\data'
em_list = np.sort(glob.glob(os.path.join(em_path, '*.xlsx')))
em = pd.read_excel(em_list[0])
em_list

In [ ]:
em

In [ ]:
input_neuron = em['Tm16'].fillna(0).values.astype(int) #replace NaNs with 0
df_em = pd.DataFrame({'fly_ID':'EM', 'cluster': range(len(input_neuron)), 'counts': input_neuron, 'method':'EM'})
df_closest_brp_all_flies = pd.concat([df_closest_brp_all_flies, df_em]) 

- The EM data has a proofreading threshold of 3 synapses. To make the two datasets comparable, we replace the synaptic counts below 3 with 0.

In [ ]:
df_closest_brp_all_flies.loc[df_closest_brp_all_flies['counts'] < 3, 'counts'] = 0
#df_closest_brp_all_flies

In [ ]:
csv_save_all_flies = os.path.join(r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm9_Tm16', 'counts_all_flies_Tm9_Tm16_em.csv')
df_closest_brp_all_flies.to_csv(csv_save_all_flies)

# Plot data

In [ ]:
singel_fly_counts = df_closest_brp_all_flies.groupby(['fly_ID', 'counts', 'method']).size()
hist_data = singel_fly_counts.to_frame('hist').reset_index()

fly_ID = [1,2,3,4,5,'1', '2', '3', '4', '5', 'EM']
norm_hist = pd.DataFrame()
for fly in fly_ID:
    curr_fly = hist_data.loc[hist_data['fly_ID'] == fly]
    curr_fly['hist_norm'] = (curr_fly['hist']/curr_fly['hist'].sum()*100)
    norm_hist = pd.concat([norm_hist, curr_fly])
#norm_hist

In [ ]:
ax = plt.figure(figsize = (11, 3))
ax = sns.barplot(data = norm_hist, x = 'counts', y = 'hist_norm', hue = 'method', palette = ['skyblue', 'maroon'], errwidth = 1.5)
ax = sns.swarmplot(data = norm_hist, x = 'counts', y = 'hist_norm', hue = 'method', dodge = True, palette = ['k', 'maroon'], size = 3)
#ax.xaxis.set_ticks(np.arange(0, 120, 1))
ax.set_xlabel('Number of brp puncta per column')
ax.set_title('Tm9 and Tm16')
ax.set_ylabel('Column Count (Percent)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='upper right')
#plt.savefig(r'C:\Users\jcornean\PhD\expansion_microscopy\data\Tm16_Tm9\Tm16_Tm9_barplot_mean_exm_em_w0.pdf')